<a href="https://colab.research.google.com/github/dheeraj-pilex/shihtzu/blob/main/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pandas transformers datasets torch scikit-learn nltk

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
import torch
from sklearn.metrics import classification_report


In [7]:
# Load your dataset
dataset = pd.read_csv("/content/Grammar Correction.csv")

# Preview the dataset
print(dataset.head())

# Drop rows with missing values in key columns
dataset = dataset.dropna(subset=['Ungrammatical Statement', 'Standard English'])

# Convert Error Type to binary labels for Grammar Detection
dataset['is_error'] = dataset['Error Type'].apply(lambda x: 1 if pd.notnull(x) else 0)

   Serial Number         Error Type              Ungrammatical Statement  \
0              1  Verb Tense Errors        I goes to the store everyday.   
1              2  Verb Tense Errors  They was playing soccer last night.   
2              3  Verb Tense Errors     She have completed her homework.   
3              4  Verb Tense Errors            He don't know the answer.   
4              5  Verb Tense Errors            The sun rise in the east.   

                       Standard English  
0           I go to the store everyday.  
1  They were playing soccer last night.  
2       She has completed her homework.  
3           He doesn't know the answer.  
4            The sun rises in the east.  


In [9]:
# Split data for detection and correction tasks
X_train, X_test, y_train, y_test = train_test_split(
    dataset['Ungrammatical Statement'], dataset['is_error'], test_size=0.2, random_state=42
)

# For correction, create input-output pairs
correction_data = dataset[['Ungrammatical Statement', 'Standard English']].reset_index(drop=True)


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset # Import Dataset class

# Load BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Tokenize data
def tokenize_data(sentences, labels, tokenizer):
    tokenized_inputs = tokenizer(sentences.to_list(), truncation=True, padding=True, max_length=128, return_tensors="pt")
    tokenized_inputs['labels'] = torch.tensor(labels.to_list())
    return tokenized_inputs

# Create a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# Apply tokenization and create Dataset objects
train_encodings = tokenize_data(X_train, y_train, tokenizer)
test_encodings = tokenize_data(X_test, y_test, tokenizer)
train_data = CustomDataset(train_encodings)  # Wrap with CustomDataset
test_data = CustomDataset(test_encodings)    # Wrap with CustomDataset

# ... (rest of your code remains the same) ...



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Load T5 model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Prepare data for T5 fine-tuning
def prepare_t5_data(df, tokenizer, max_length=128):
    inputs = tokenizer(
        ["grammar correction: " + text for text in df['Ungrammatical Statement']],
        truncation=True, padding="max_length", max_length=max_length, return_tensors="pt"
    )
    targets = tokenizer(
        df['Standard English'].to_list(),
        truncation=True, padding="max_length", max_length=max_length, return_tensors="pt"
    )
    return inputs.input_ids, targets.input_ids

input_ids, target_ids = prepare_t5_data(correction_data, t5_tokenizer)

# Fine-tune the T5 model
class T5Dataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, target_ids):
        self.input_ids = input_ids
        self.target_ids = target_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "labels": self.target_ids[idx]
        }

train_dataset = T5Dataset(input_ids[:int(len(input_ids)*0.8)], target_ids[:int(len(target_ids)*0.8)])
eval_dataset = T5Dataset(input_ids[int(len(input_ids)*0.8):], target_ids[int(len(target_ids)*0.8):])

training_args = TrainingArguments(
    output_dir="./t5_results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
)

trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
trainer.train()

# Test correction model
def test_t5_model(model, tokenizer, sentences):
    for sentence in sentences:
        input_ids = tokenizer("grammar correction: " + sentence, return_tensors="pt").input_ids
        outputs = model.generate(input_ids)
        corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Original: {sentence}")
        print(f"Corrected: {corrected_text}")

test_sentences = ["She go to school", "I likes apples"]
test_t5_model(t5_model, t5_tokenizer, test_sentences)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,0.165393
2,No log,0.095112
3,0.192500,0.092534


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Original: She go to school
Corrected: She goes to school
Original: I likes apples
Corrected: Ich mag pfeln.


In [12]:
\\# Save models
bert_model.save_pretrained("./bert_grammar_detection")
t5_model.save_pretrained("./t5_grammar_correction")